In [1]:
%load_ext autoreload
%autoreload
%matplotlib inline

import os
import sys
parentdir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parentdir)

from newscrawler import NewsCrawler
from IPython.display import display
from IPython.core.debugger import set_trace

# 0. 초기화

In [2]:
crawler = NewsCrawler(storage='bigquery')

checking bigquery storage... done (27.87 seconds)
we have total 420850 articles (358561 downloaded, 62289 trashed)


# 1. URL을 모은다

In [3]:
collect_result = crawler.collect(); display(*collect_result)

100.00% (94.54 seconds): Collecting URLs... euronews            

100.00% (0.12 seconds): Selecting URLs... cryptonews          

,collected,collected_unique,selected
all,25990,25923,528
reuters,1086,1086,97
yahoo,387,387,68
euronews,1046,1046,50
dailymail,2927,2927,42
...,...,...,...
realclearpolitics,293,268,0
huffpost,64,64,0
nationalreview,743,742,0
news24,1213,1211,0


,url,pubs,actual_pub
0,https://cnn.com/2019/10/30/business/grocery-de...,"cnn, axios",cnn
1,https://politico.com/magazine/story/2019/11/05...,"politico, realclearpolitics",politico
2,https://politico.com/magazine/story/2019/10/30...,"politico, realclearpolitics",politico
3,https://politico.com/news/2019/11/06/joe-biden...,"politico, realclearpolitics",politico
4,https://politico.com/news/2019/11/06/jeff-sess...,"politico, realclearpolitics",politico
...,...,...,...
60,https://nbcnews.com/politics/trump-impeachment...,"msnbc, nbcnews",nbcnews
61,https://nbcnews.com/nightly-news/video/top-fem...,"msnbc, nbcnews",nbcnews
62,https://nbcnews.com/nightly-news/video/bloombe...,"msnbc, nbcnews",nbcnews
63,https://nbcnews.com/politics/donald-trump/inte...,"msnbc, nbcnews",nbcnews


# 2. 해당 URL의 뉴스를 크롤링한다

In [4]:
crawl_result = crawler.crawl(); crawl_result

100.00% (222.21 seconds): Crawling... yahoo               



collect                                crawl              
                  collected collected_unique selected downloaded trashed total
all                   25990            25923      528        320     208   528
reuters                1086             1086       97         52      45    97
yahoo                   387              387       68         59       9    68
euronews               1046             1046       50          1      49    50
dailymail              2927             2927       42         37       5    42
...                     ...              ...      ...        ...     ...   ...
realclearpolitics       293              268        0          0       0     0
huffpost                 64               64        0          0       0     0
nationalreview          743              742        0          0       0     0
news24                 1213             1211        0          0       0     0
cryptonews              333              333        0          0       0     0

[74 rows x 6 columns]

# 3. 뉴스를 저장한다

In [5]:
crawler.record()

Recording to bigquery storage... done (17.10 seconds)


# 4. 결과보고

In [7]:
crawler.report(using='sendgrid')

202


In [21]:
import requests, json

In [24]:
json.loads(requests.get('https://us-central1-global-news-crawl.cloudfunctions.net/test').text)

{'a': 1, 'b': 2, 'c': 3}

In [5]:
import os
import sys
import json
import requests
import pickle
import gzip
import pandas as pd
import pandas_gbq as gbq
from pathlib import Path
from google.oauth2 import service_account
from google.cloud import bigquery
from IPython.core.debugger import set_trace
from tqdm.auto import tqdm
tqdm.pandas()
from newscrawler import accounts

C:\Users\infomax\Anaconda3\lib\site-packages\tqdm\std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [6]:
proj = 'global-news-crawl'
table_downloaded = 'news_dataset.downloaded'
table_trashed = 'news_dataset.trashed'
credentials = service_account.Credentials.from_service_account_info(accounts.bigquery_account)

In [82]:
qry_0 = 'SELECT id, url, title, downloaded_at FROM `{}`'.format(proj + '.' + table_trashed)
ids_trashed = gbq.read_gbq(qry_0, project_id=proj, credentials=credentials)

In [86]:
qry_1 = 'SELECT title FROM `{}` GROUP BY title HAVING COUNT(title)>1'.format(proj + '.' + table_downloaded)
dumps = gbq.read_gbq(qry_1, project_id=proj, credentials=credentials)

In [88]:
dumps#['time'] = dumps.downloaded_at.str[:13]; dumps

,title
0,Meet the rebel Thai composer taking music to u...
1,"Emotional intelligence: What is it, and why do..."
2,The private world of London's Traveler community
3,'Elephant Queen' in Kenya: See incredible pict...
4,Storm Area 51: World-famous DJ performs to cro...
...,...
37795,"IndiGo says systems down, likely to hit operat..."
37796,France's EDF buys electric vehicle charging po...
37797,Three killed as Iraqi protesters try to break ...
37798,IMF cuts Bosnia's growth forecast to 2.8% in 2...


In [55]:
dumps[dumps.time=='2019-11-07 12']

,id,url,title,downloaded_at,time
0,5f2471fb9b88183e971f37021f3f668077fd7312,https://afr.com/companies/financial-services/c...,"Cut red tape to boost economy, say CFOs",2019-11-07 12:32:05.873639+00:00,2019-11-07 12
1,92e89fac2e9db8fa0143bfa60dfa782c5cf6e5f6,https://afr.com/property/commercial/lendlease-...,Lendlease sells Adelaide mall stake for $670m,2019-11-07 12:32:17.364064+00:00,2019-11-07 12
2,d3d6e484d0dc661efbc719b70914662b2fa2b0eb,https://bbc.co.uk/news/av/uk-england-hereford-...,From Morrisons to Fatboy Slim for world trip l...,2019-11-07 12:33:02.871903+00:00,2019-11-07 12
3,6f7dc170215e1cc1fd6186098fe5e55937265b11,https://bbc.com/news/entertainment-arts-50330537,James Dean CGI 'casting' angers Avengers star ...,2019-11-07 12:32:49.898174+00:00,2019-11-07 12
4,84d0707511509e359cd29560de6edfe2870f6c4b,https://cbc.ca/news/world/dozens-still-missing...,Dozens still missing after deadly attack near ...,2019-11-07 12:32:41.021597+00:00,2019-11-07 12
...,...,...,...,...,...
768,ae7d2916d8cd8c9c08f5082044b836e5f42e2f47,https://businessinsider.com/elon-musk-mulling-...,Elon Musk said he will look into launching a '...,2019-11-07 12:32:47.273799+00:00,2019-11-07 12
769,938693e23e1a603e6f7c6e888cd7f6d4c0cf09eb,https://businessinsider.com/anonymous-white-ho...,White House aides were certain Pence would sup...,2019-11-07 12:35:32.123105+00:00,2019-11-07 12
770,dc3c33e123cea10674e55cd592b60949bc897556,https://nordic.businessinsider.com/australian-...,An Australian man allegedly downloaded softwar...,2019-11-07 12:35:21.703648+00:00,2019-11-07 12
771,cca1a65f6caa14ee2144a8d154242839f68f55b6,https://washingtontimes.com/news/2019/nov/7/as...,Assembly weighs expanding birth control access...,2019-11-07 12:33:37.280983+00:00,2019-11-07 12


In [97]:
ids_trashed[ids_trashed.duplicated('title')]

,id,url,title,downloaded_at
3,13409b2ddf1b1019bd004f49dc8adb2db64156a4,https://jp.reuters.com/article/eurozone-econom...,欧州委、ユーロ圏成長率予想引き下げ 貿易摩擦や地政学リスク,2019-11-07 11:22:20.508525+00:00
13,70560d2300728e3d30fa05e133f8dda39ee90b9d,https://jp.reuters.com/article/bank-permata-m-...,三井住友やＯＣＢＣ、プルマタ銀買収合戦に参加へ＝関係筋,2019-11-07 11:22:41.804174+00:00
15,65afa0adf42d9d8c4426b759bba40adbed28cb45,https://jp.reuters.com/article/idJPKBN1XH1GX,欧州委、ユーロ圏成長率予想引き下げ 貿易摩擦や地政学リスク,2019-11-07 11:22:47.631572+00:00
16,d220ef201ce37564637823a7b1fc36164e7251fd,https://jp.reuters.com/article/ecb-policy-holz...,ＥＣＢ、できるだけ早くマイナス金利脱却すべき＝オーストリア中銀総裁,2019-11-07 11:22:48.270992+00:00
27,2e0733eed54a7e86d5815ae94d3d3ab7759f80ca,https://newslink.reuters.com/article/us-alital...,"Business & Financial News, U.S & International...",2019-11-07 11:22:21.824642+00:00
...,...,...,...,...
60143,46d0e93589165978f46a69d9b3ec815ed37096df,https://newslink.reuters.com/article/us-china-...,"Business & Financial News, U.S & International...",2019-11-07 07:23:50.548551+00:00
60144,ce5ebb56f3c5bd7cae1f9aee2c4b41512f188994,https://newslink.reuters.com/article/us-japan-...,"Business & Financial News, U.S & International...",2019-11-07 07:23:54.058081+00:00
60145,a3db48bd3220d246cfe5c0c4f0f7e1c8e2f4ce08,https://newslink.reuters.com/article/us-rakute...,"Business & Financial News, U.S & International...",2019-11-07 07:23:56.388862+00:00
60146,078b2df640984544f9850479cc9b4b846275c518,https://newslink.reuters.com/article/us-haneda...,"Business & Financial News, U.S & International...",2019-11-07 07:23:59.233480+00:00


In [103]:
list(ids_trashed[ids_trashed.title=='欧州委、ユーロ圏成長率予想引き下げ 貿易摩擦や地政学リスク'].title)

['欧州委、ユーロ圏成長率予想引き下げ 貿易摩擦や地政学リスク',
 '欧州委、ユーロ圏成長率予想引き下げ 貿易摩擦や地政学リスク',
 '欧州委、ユーロ圏成長率予想引き下げ 貿易摩擦や地政学リスク']